In [1]:
import pandas as pd
import os
import requests

from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
# Suppress DtypeWarning
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Root of project
root_dir = os.path.join("analysis_data")
os.makedirs(root_dir, exist_ok=True)

# Raw data from api
data_dir = os.path.join(root_dir, "data")
os.makedirs(data_dir, exist_ok=True)

# Formatted CSV data
csv_dir = os.path.join(root_dir, "csv")
os.makedirs(csv_dir, exist_ok=True)

# Data split by location
location_dir = os.path.join(root_dir, "location")
os.makedirs(location_dir, exist_ok=True)

In [ ]:
def get_data(start: str, end: str):
    data = requests.get(
        f"https://ilm2.site.dustmonitoring.nl/download?from={start}&to={end}&interval=600&align=1&type=csv-semicolon&p=531&p=521&p=542&p=543&p=553&p=544&p=545&p=532&p=533&p=554&p=534&p=535&p=546&p=536&p=556&p=522&p=557&p=547&p=549&p=524&p=537&p=525&p=526&p=539&p=551&p=540&p=558&p=527&p=528&p=529&p=530&p=560&p=561&p=562&p=563&p=564&p=565&p=566&p=567&p=568&p=569&p=570&p=571&p=574&p=575&p=576&p=577&p=578&s=10&s=11&s=128&s=129&s=130&s=145&s=146"
    )
    return data.text

_date = date(2020, 11, 1)

while True:
    start_date = _date
    end_date = _date + relativedelta(months=2)
    data = get_data(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    file_path = os.path.join(data_dir, f"data_{start_date.strftime('%Y-%m-%d')}.csv")

    with open(file_path, "+w") as file:
        file.write(data)

    _date = end_date
    if _date > date(2024, 3, 1):
        break

In [ ]:
dfs = []
for file in os.listdir(data_dir):

    file_path = os.path.join(data_dir, file)

    if not os.path.isfile(file_path):
        continue
    df = pd.read_csv(file_path, index_col=False, sep=";")

    # Get the values of the first 2 rows
    header_string = df.iloc[:2].values
    row_1 = [row.split(".")[0] for row in df.columns.tolist()]
    row_2 = header_string[0]

    # Merge these into the new column names
    new_columns = []
    for row1, row2 in zip(row_1, row_2):
        row1 = row1.replace("Unnamed: ", "")
        new_columns.append(f"{row1}-{row2}")
    # Remove the used rows
    df = df.iloc[2:]
    # Set new column names
    df.columns = new_columns
    
    csv_file_path = os.path.join(csv_dir, file)
    df.to_csv(csv_file_path, index=False, index_label=False)

In [ ]:
# Get all monthly datasets
dfs = []
for file in os.listdir(csv_dir):
    csv_file_path = os.path.join(csv_dir, file)
    df = pd.read_csv(csv_file_path, index_col=False)
    dfs.append(df)

# Join datasets togather into one
df = pd.concat(dfs, ignore_index=True)
df.shape

In [ ]:
for location in range(1, 60):
    location = str(location)
    if len(location) < 2:
        location = '0' + location
    
    location_columns = ['0-Tijd', '1-Tijd']
    for column in df.columns:
        if location in column:
            location_columns.append(column)

    if len(location_columns) > 2:
        df_temp = df[location_columns]
        location_csv_path = os.path.join(location_dir, f"I{location}.csv")
        df_temp.to_csv(location_csv_path, index=False, index_label=False)

In [ ]:
results = []
for location in range(1, 60):
    location = str(location)
    if len(location) < 2:
        location = '0' + location
    location_csv_path = os.path.join(location_dir, f"I{location}.csv")
    if not os.path.isfile(location_csv_path):
        continue
    df = pd.read_csv(location_csv_path)
    df = df.iloc[df[df.columns[2]].first_valid_index():]
    df = df.iloc[:df[df.columns[2]].last_valid_index()]

    total = df.shape[0]
    missing = df[df.columns[2]].isnull().sum()
    perc = round(missing/total*100, 2)

    obj = {
        "location": location,
        "start": df["0-Tijd"][df["0-Tijd"].first_valid_index()],
        "end": df["0-Tijd"][df["0-Tijd"].last_valid_index()],
        "total rows": total,
        "missing rows": missing,
        "percentage missing": f"{round(perc, 2)} %"
    }
    results.append(obj)

In [ ]:
df_res = pd.DataFrame.from_records(results)
df_res